In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('Set2')

#한글폰트 가져오기
from matplotlib import rc
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False #음수에 (-)표현

# Warning 메세지를 뜨지 않게 해줌
import warnings
warnings.filterwarnings('ignore')

from pyproj import Proj, transform
from scipy.spatial import KDTree
from haversine import haversine

In [23]:
df=pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0525/df_17_19.csv",encoding='cp949')
df.head()

,구급보고서번호,등록완료구분명,환자발생유형구분명,교통사고명,교통사고기타명,소방서명,서센터명,소방지역대명,구급활동구분명,헬리콥터펌뷸런스유무명,...,현장거리2,재이송사유2,이송분류명,이송분류사유,시간단위기온,시간단위강수량,시간단위풍속,시간단위풍향,시간단위습도,시간단위적설량
0,20174501102M00002,등록완료,질병외,동승자,NaN,전주덕진소방서,팔복119안전센터,NaN,특수일반,NaN,...,NaN,NaN,이송,NaN,2.2,NaN,1.9,140.0,86.0,NaN
1,20174502106M00003,등록완료,질병외,운전자,NaN,전주완산소방서,평화119안전센터,NaN,특수일반,NaN,...,NaN,NaN,이송,NaN,2.2,NaN,1.9,140.0,86.0,NaN
2,20174502108M00002,등록완료,질병외,동승자,NaN,전주완산소방서,서부119안전센터,NaN,특수일반,NaN,...,NaN,NaN,이송,NaN,2.2,NaN,1.9,140.0,86.0,NaN
3,20174502108M00004,등록완료,질병외,운전자,NaN,전주완산소방서,서부119안전센터,NaN,특수일반,NaN,...,NaN,NaN,이송,NaN,2.2,NaN,1.9,140.0,86.0,NaN
4,20174502102M00005,등록완료,질병외,오토바이사고,NaN,전주완산소방서,효자119안전센터,NaN,특수전문,펌뷸런스,...,NaN,NaN,이송,NaN,2.0,NaN,1.0,160.0,87.0,NaN


In [24]:
# 나이 
df=df[(df['환자연령']<130)&(df['환자연령']>0)]

In [25]:
# 위치정보
df.dropna(subset=['위치정보X','위치정보Y'],inplace=True)

## 밀도

In [26]:
# 밀도 계산
def calculate_density(df, x_col='위치정보X', y_col='위치정보Y', radius=100): #radius (float): 거리 반경 (기본값 100 미터).

    # UTM 좌표에서 경위도 좌표로 변환
    utm_proj = Proj(init = 'epsg:5181')
    latlon_proj = Proj(init='epsg:4326')
    
    df = df.dropna(subset=[x_col, y_col]) #na를 데이터 프레임에서 아예 없애버림
    lon, lat = transform(utm_proj, latlon_proj, df[x_col].values, df[y_col].values)
    df['경도'] = lon
    df['위도'] = lat
    
    # KDTree를 이용한 거리 계산 및 밀도 추정
    coords = np.vstack([lon, lat]).T
    tree = KDTree(coords)
    
    # 각 점 주변 반경 내 점의 개수를 센다
    densities = tree.query_ball_point(coords, r=radius / 111000)  # 111000 미터는 1도 위도
    density = [len(points) for points in densities]
    
    return density

In [27]:
df['밀도'] = calculate_density(df)

In [30]:
# UTM좌표를 WGS84로 변환
proj_UTM = Proj(init = 'epsg:5181')
proj_WGS84 = Proj(init='epsg:4326')
df['경도'],df['위도'] = transform(proj_UTM,proj_WGS84,df['위치정보X'],df['위치정보Y'])


from sklearn.neighbors import KernelDensity

data = df[['위도','경도']]

# 커널 밀도 추정을 위한 100미터에 해당하는 밴드폭 설정
bandwidth = 0.0009

kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
kde.fit(data)

# 각 데이터 포인트의 로그 밀도 값 계산
log_density_values = kde.score_samples(data)

# 로그 밀도 값을 지수 함수로 변환하여 실제 밀도 값 계산
density_values = np.exp(log_density_values)

df['밀도_kde'] = density_values

## 변수생성

### 가장 가까운 학교로부터 거리

In [28]:
df_school = pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0525/school_17_19.csv",encoding='cp949')

# UTM좌표를 WGS84로 변환
proj_UTM = Proj(init = 'epsg:5181')
proj_WGS84 = Proj(init='epsg:4326')

def UTM_to_WGS(df):
    X,Y = transform(proj_UTM,proj_WGS84,df['위치정보X'],df['위치정보Y'])
    return pd.DataFrame({'위도':Y,'경도':X},index = df.index)

def find_nearest_school(df):
    results = [] #결과를 담을 객체 생성

    for _, pedestrian in UTM_to_WGS(df).iterrows(): # 보행자의 위치별로 시행
        #가까운 학교와 최소 거리 초기화
        min_distance = float('inf')
        nearest_school = None

        for _, school in df_school.iterrows(): #보행자별 각 학교 시행
            school_location = (school['위도'], school['경도'])
            pedestrian_location = (pedestrian['위도'], pedestrian['경도'])

            #거리계산
            distance = haversine(pedestrian_location, school_location, unit='m') #거리 기준 변환가능
            
            if distance < min_distance: #최소거리와 학교를 담음
                min_distance = distance
                nearest_school = school
        #보행자별 시행을 마치고 result에 값을 담음
        results.append({
                '보행자_위도': pedestrian['위도'],
                '보행자_경도': pedestrian['경도'],
                '가장_가까운_학교': nearest_school['학교명'],  # 학교명이 있는 열로 변경
                '가장_가까운_학교_위도': nearest_school['위도'],
                '가장_가까운_학교_경도': nearest_school['경도'],
                '학교로부터_거리': min_distance
            })
        
    return pd.DataFrame(results,index=df.index)

In [29]:
dist_from_school = find_nearest_school(df)
dist_from_school.sort_values(by='학교로부터_거리')

,보행자_위도,보행자_경도,가장_가까운_학교,가장_가까운_학교_위도,가장_가까운_학교_경도,학교로부터_거리
267,35.838585,127.150885,전주동중학교,35.838585,127.150885,0.000679
266,35.838585,127.150885,전주동중학교,35.838585,127.150885,0.000679
452,35.825313,127.132008,전주진북초등학교,35.825313,127.132008,0.000819
4788,35.803250,127.122752,전주효자초등학교,35.803250,127.122752,0.002956
3065,35.837437,127.111161,전주여울초등학교,35.837437,127.111161,0.003880
...,...,...,...,...,...,...
7407,35.585452,126.825936,전주중인초등학교,35.771331,127.097938,32106.108458
6268,35.771599,127.613535,전주대성초등학교,35.797597,127.177787,39412.283450
6270,35.771599,127.613535,전주대성초등학교,35.797597,127.177787,39412.283450
6269,35.771599,127.613535,전주대성초등학교,35.797597,127.177787,39412.283450


### 어린이 보호구역 여부
스쿨존  
 초등학교 및 유치원 주출입문에서 반경 300m 이내의 주통학로를 보호구역으로 지정하여 교통안전시설물 및 도로부속물 설치로 학생들의 안전한 통학공간을 확보하여 교통사고를 예방하기 위한 제도를 말한다. 1995년 도로교통법에 의해 도입되었으며, 그해 '어린이 보호구역의 지정 및 관리에 관한 규칙'이 제정되었다.
[네이버 지식백과] 스쿨존 (시사상식사전, pmg 지식엔진연구소)

In [31]:
dist_from_school['스쿨존여부(300m)'] = (dist_from_school['학교로부터_거리'] <= 300).astype(int)

In [32]:
dist_from_school.head()

,보행자_위도,보행자_경도,가장_가까운_학교,가장_가까운_학교_위도,가장_가까운_학교_경도,학교로부터_거리,스쿨존여부(300m)
0,35.821035,127.103056,전주홍산초등학교,35.820672,127.098952,372.211351,0
1,35.821035,127.103056,전주홍산초등학교,35.820672,127.098952,372.211351,0
2,35.821035,127.103056,전주홍산초등학교,35.820672,127.098952,372.211351,0
3,35.821035,127.103056,전주홍산초등학교,35.820672,127.098952,372.211351,0
4,35.805148,127.110476,전주우전초등학교,35.805973,127.107375,294.260937,1


In [ ]:
### 버스정류장 유무

In [33]:
import numpy as np
from scipy.spatial.distance import cdist
from pyproj import Transformer
from haversine import haversine

In [34]:
df_bus=pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/버스정류소/전주시 정류소 현황_20210426.csv",encoding='cp949')
df_bus.head()
df_bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   설치 지점명  569 non-null    object 
 1   지역(구)   569 non-null    object 
 2   지역(동)   534 non-null    object 
 3   세부주소    569 non-null    object 
 4   행정동     502 non-null    object 
 5   위도      569 non-null    float64
 6   경도      569 non-null    float64
dtypes: float64(2), object(5)
memory usage: 31.2+ KB


In [35]:
def find_nearest_bus(df):
    results = [] #결과를 담을 객체 생성

    for _, pedestrian in df[['위도','경도']].iterrows(): # 보행자의 위치별로 시행
        #가까운 학교와 최소 거리 초기화
        min_distance = float('inf')
        nearest_bus = None

        for _, bus in df_bus.iterrows(): #보행자별 각 정류장 시행
            bus_location = (bus['위도'], bus['경도'])
            pedestrian_location = (pedestrian['위도'], pedestrian['경도'])

            #거리계산
            distance = haversine(pedestrian_location, bus_location, unit='m') #거리 기준 변환가능
            
            if distance < min_distance: #최소거리와 정류장을 담음
                min_distance = distance
                nearest_bus = bus
        #보행자별 시행을 마치고 result에 값을 담음
        results.append({
                '보행자_위도': pedestrian['위도'],
                '보행자_경도': pedestrian['경도'],
                '가장_가까운_버스정류장': nearest_bus['설치 지점명'],  
                '가장_가까운_정류장_위도': nearest_bus['위도'],
                '가장_가까운_정류장_경도': nearest_bus['경도'],
                '버스정류장으로부터_거리': min_distance
            })
        
    return pd.DataFrame(results,index=df.index)

In [36]:
df2=find_nearest_bus(df)
df2.head()

,보행자_위도,보행자_경도,가장_가까운_버스정류장,가장_가까운_정류장_위도,가장_가까운_정류장_경도,버스정류장으로부터_거리
0,35.821035,127.103056,유로병원,35.819202,127.104073,223.408336
1,35.821035,127.103056,유로병원,35.819202,127.104073,223.408336
2,35.821035,127.103056,유로병원,35.819202,127.104073,223.408336
3,35.821035,127.103056,유로병원,35.819202,127.104073,223.408336
4,35.805148,127.110476,효자롯데아파트,35.805875,127.112852,228.991748


In [38]:
df2['버스_정류장_유무(10m)'] = df2['버스정류장으로부터_거리']<=10
df2['버스_정류장_유무(10m)'].astype(int)

0       0
1       0
2       0
3       0
4       0
       ..
8374    0
8375    0
8376    0
8377    0
8378    0
Name: 버스_정류장_유무(10m), Length: 8269, dtype: int32

In [40]:
df2['버스_정류장_유무(10m)'].value_counts()

버스_정류장_유무(10m)
False    8204
True       65
Name: count, dtype: int64

In [48]:
df2.to_csv('C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0525/bus_17_19.csv',index=False)

In [ ]:
### 신호등

In [37]:
import pandas as pd
from geopy.distance import geodesic

In [43]:
# light 데이터프레임 생성
light = pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/신호등/전라북도_신호등 현황 제공_20211221/전라북도 신호등 현황_전주시(보완).csv")
light = light[light['도로형태']==2]
light.iloc[:,:11]
# '신호등 관리 번호'로 그룹화하여 위도와 경도의 평균을 구함
cross = light.groupby('신호등관리번호').agg({'위도': 'mean', '경도': 'mean'}).reset_index()

In [44]:
# 거리 계산 함수
def calculate_distance(row):
    light_location = (row['위도'], row['경도'])
    light_group = light[light['신호등관리번호'] == row['신호등관리번호']]
    distances = []
    for _, l in light_group.iterrows():
        cross_location = (row['위도'], row['경도'])
        distance = geodesic(light_location, (l['위도'], l['경도'])).meters
        distances.append(distance)
    return {
        '평균거리': sum(distances) / len(distances),
        '최대거리': max(distances)
    }

# '평균거리'와 '최대거리' 계산
cross[['평균거리', '최대거리']] = cross.apply(calculate_distance, axis=1, result_type='expand')

print(cross)

    신호등관리번호         위도          경도       평균거리       최대거리
0         1  35.849327  127.160470  21.358043  25.077371
1        10  35.841265  127.134568  14.578311  14.578312
2       100  35.823053  127.152621  16.644848  18.608374
3       102  35.810977  127.161225  13.788804  16.733525
4       103  35.808996  127.162229  15.352714  19.746543
..      ...        ...         ...        ...        ...
742      96  35.819666  127.153992  18.513234  33.730161
743      97  35.818268  127.154644  10.398655  15.644873
744    97.A  35.817724  127.154801  10.128300  24.606183
745      98  35.816615  127.154648  15.665618  19.814798
746      99  35.823141  127.151201  14.006119  19.257785

[747 rows x 5 columns]


In [45]:
# 교차로 여부
from haversine import haversine

def find_nearest_cross(df):
    results = [] #결과를 담을 객체 생성

    for _, pedestrian in df[['위도','경도']].iterrows(): # 보행자의 위치별로 시행
        #가까운 학교와 최소 거리 초기화
        min_distance = float('inf')
        nearest_cross = None

        for _, cross_ in cross.iterrows(): #보행자별 각 학교 시행
            cross_location = (cross_['위도'], cross_['경도'])
            pedestrian_location = (pedestrian['위도'], pedestrian['경도'])

            #거리계산
            distance = haversine(pedestrian_location, cross_location, unit='m') #거리 기준 변환가능
            
            if distance < min_distance: #최소거리와 학교를 담음
                min_distance = distance
                nearest_cross = cross_
        #보행자별 시행을 마치고 result에 값을 담음
        results.append({
                '환자_위도': pedestrian['위도'],
                '환자_경도': pedestrian['경도'],
                '가장_가까운_교차로': nearest_cross['신호등관리번호'],  # 학교명이 있는 열로 변경
                '가장_가까운_교차로_위도': nearest_cross['위도'],
                '가장_가까운_교차로_경도': nearest_cross['경도'],
                '교차로로부터_거리': min_distance,
                '평균거리':nearest_cross['평균거리'],
                '최대거리':nearest_cross['최대거리']
            })
        
    return pd.DataFrame(results,index=df.index)

In [46]:
result_cross = find_nearest_cross(df)
result_cross

,환자_위도,환자_경도,가장_가까운_교차로,가장_가까운_교차로_위도,가장_가까운_교차로_경도,교차로로부터_거리,평균거리,최대거리
0,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692
1,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692
2,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692
3,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692
4,35.805148,127.110476,240,35.805106,127.110542,7.613715,14.827280,23.838310
...,...,...,...,...,...,...,...,...
8374,35.809422,127.114630,304,35.810690,127.114797,141.784459,6.243022,9.944013
8375,35.836533,127.110014,346.A,35.836896,127.112163,197.855067,5.708704,9.315141
8376,35.793392,127.132797,31,35.794117,127.132374,89.199293,10.741594,18.775484
8377,35.816484,127.123121,22,35.816531,127.122977,14.022138,23.122752,28.261744


In [47]:
result_cross.sort_values(by = '교차로로부터_거리')

,환자_위도,환자_경도,가장_가까운_교차로,가장_가까운_교차로_위도,가장_가까운_교차로_경도,교차로로부터_거리,평균거리,최대거리
7748,35.823052,127.152623,100,35.823053,127.152621,0.236540,16.644848,18.608374
4536,35.824984,127.131964,176,35.824982,127.131965,0.249628,12.830818,21.784192
4537,35.824984,127.131964,176,35.824982,127.131965,0.249628,12.830818,21.784192
5064,35.819050,127.112200,281,35.819044,127.112203,0.723108,18.920286,25.240055
5063,35.819050,127.112200,281,35.819044,127.112203,0.723108,18.920286,25.240055
...,...,...,...,...,...,...,...,...
7407,35.585452,126.825936,268,35.790686,127.065955,31475.121159,8.303905,14.663486
6270,35.771599,127.613535,378,35.778897,127.198574,37444.209539,10.288881,16.175581
6269,35.771599,127.613535,378,35.778897,127.198574,37444.209539,10.288881,16.175581
6268,35.771599,127.613535,378,35.778897,127.198574,37444.209539,10.288881,16.175581


In [49]:
# '교차로여부' 열 생성 및 조건에 따라 값 할당
result_cross['교차로여부'] = (result_cross['교차로로부터_거리'] <= result_cross['최대거리']).astype(int)
result_cross

,환자_위도,환자_경도,가장_가까운_교차로,가장_가까운_교차로_위도,가장_가까운_교차로_경도,교차로로부터_거리,평균거리,최대거리,교차로여부
0,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692,0
1,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692,0
2,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692,0
3,35.821035,127.103056,286,35.821038,127.101888,105.385363,15.601503,19.970692,0
4,35.805148,127.110476,240,35.805106,127.110542,7.613715,14.827280,23.838310,1
...,...,...,...,...,...,...,...,...,...
8374,35.809422,127.114630,304,35.810690,127.114797,141.784459,6.243022,9.944013,0
8375,35.836533,127.110014,346.A,35.836896,127.112163,197.855067,5.708704,9.315141,0
8376,35.793392,127.132797,31,35.794117,127.132374,89.199293,10.741594,18.775484,0
8377,35.816484,127.123121,22,35.816531,127.122977,14.022138,23.122752,28.261744,1


In [50]:
result_cross.to_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0525/cross_17_19.csv",index=False)